# 内容说明
1. 实验目标  
算法内容：手动实现 BPR+MF 贝叶斯个性化排名的推荐系统算法  
数据来源：使用 RecBole 中提供的ml-100k的数据  
误差标准：使用 RMSE 均方根误差作为误差的衡量标准  
  
2. 本ipynb内容  
由于计算后验概率值以及每轮的参数更新时间复杂度太高，为n_user\*n_item\*n_item，只使用了50个user和50个item的数据  
未划分训练集和测试集，未编写类  


In [1]:
import pandas as pd
import torch
import math

## 1. 读取数据
使用pandas库下的read_csv来读取几个文件，只需使用三个文件，.user .item和.inter，剩下的两个.kg和.link用不到

In [2]:
user = pd.read_csv('../dataset/ml-100k/ml-100k.user', delimiter='\t', engine='python')
user = user[:50]  # 只取50个user做下面实验
user

,user_id:token,age:token,gender:token,occupation:token,zip_code:token
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [3]:
item = pd.read_csv('../dataset/ml-100k/ml-100k.item', delimiter='\t', engine='python')
item = item[:50]  # 只取50个item做下面实验
item

,item_id:token,movie_title:token_seq,release_year:token,class:token_seq
0,1,Toy Story,1995,Animation Children's Comedy
1,2,GoldenEye,1995,Action Adventure Thriller
2,3,Four Rooms,1995,Thriller
3,4,Get Shorty,1995,Action Comedy Drama
4,5,Copycat,1995,Crime Drama Thriller
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao),1995,Drama
6,7,Twelve Monkeys,1995,Drama Sci-Fi
7,8,Babe,1995,Children's Comedy Drama
8,9,Dead Man Walking,1995,Drama
9,10,Richard III,1995,Drama War


In [4]:
inter = pd.read_csv('../dataset/ml-100k/ml-100k.inter', delimiter='\t', engine='python')
inter

,user_id:token,item_id:token,rating:float,timestamp:float
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## 2. 构建矩阵与列表
1. 评分矩阵 rating_mat  
行数=user数+1，列数=item数+1，初始化为0，行列数目都加1是因为user和item都从1开始编号  
2. pos_neg_item列表  
作用：记录每个用户给哪些item评分，哪些item没有评分  
pos_neg_item：为三层嵌套列表。大列表为pos_neg_item；有n_user个中列表，每个中列表对应一个user的评分情况；每个中列表中有两个小列表，分别是该user已经评分和没有评分的item编号  
获取方法：遍历一次评分矩阵，根据行号确定中列表索引，根据取值是否为0确定小列表索引  
3. 用户矩阵 user_mat  
行数=user数+1，列数=f，每一行为一个用户的f维向量，即该用户的待学习参数
4. 物品矩阵item_mat  
行数=item数+1，列数=f，每一行为一个物品的f维向量，即该物品的待学习参数

In [6]:
# 创建评分矩阵
n_user = 50
n_item = 50
rating_mat = torch.zeros(n_user+1, n_item+1)
print(rating_mat.size())

df = pd.DataFrame(inter)
for index, row in df.iterrows():
    if (row['user_id:token']<=50) & (row['item_id:token']<=50):  # 判断是否是编号前50的user以及item
        rating_mat[row['user_id:token']][row['item_id:token']] = row['rating:float']

torch.Size([51, 51])


In [7]:
# 创建pos_neg_item列表，即用户的已评和未评item编号列表
pos_neg_list = []
for i in range(n_user+1):  # 初始化为空，+1因为user从1开始编号
    pos_neg_list.append([[],[]])

# 遍历评分矩阵rating_mat，填充pos_neg_item列表
for u in range(1, n_user+1):  # user和item都是从1开始编号的
    for i in range(1, n_item+1):
        if rating_mat[u][i] != 0:
            pos_neg_list[u][0].append(i)  # 已评分，item编号记录在user的第一个列表
        else:
            pos_neg_list[u][1].append(i)  # 未评分，item编号记录在user的第二个列表

In [8]:
f = 10  # 映射的空间维度，可以调整

# 创建user矩阵与item矩阵
user_mat = torch.rand(n_user+1, f)
item_mat = torch.rand(n_item+1, f)
print(user_mat.size())
#print(user_mat, '\n')
print(item_mat.size())
#print(item_mat, '\n')

torch.Size([51, 10])
torch.Size([51, 10])


## 3. 定义最大化目标
为了检验梯度下降时是否朝着最大化后验概率的方向，每次迭代完成之后，可以计算对数后验函数的值来检验  
最大化目标即是对数后验估计函数  
定义后验函数值时，在公式前取了相反数，不然数值前面总有负号  
最大化相应的变为最小化，即最小化负的对数后验函数

In [11]:
def compute_loss(P, Q, R, pos_neg_list, lmbda):
    
    n_P = P.size()[0]
    n_Q = Q.size()[0]
    
    # 提前计算好参数二范数，防止下面三重循环中重复计算
    P_norm = 0  # 总的user向量范数
    Q_norm = 0  # 总的item向量范数
    for u in range(1, n_P):
        pu = P[u]
        P_norm += torch.norm(pu)**2
    for i in range(1, n_Q):
        qi = Q[i]
        Q_norm += torch.norm(qi)**2
    
    # 初始化后验概率值
    loss = lmbda*(P_norm + Q_norm)
    
    # 计算预测的评分值矩阵
    R_pred = torch.matmul(P, Q.t())
    
    # 计算对数后验概率值
    n = 0
    for u in range(1, n_P):
        for i in pos_neg_list[u][0]:
            for j in pos_neg_list[u][1]:
                n += 1
                xuij = R_pred[u][i] - R_pred[u][j]
                sigmod = 1/(1 + math.exp(-xuij))
                ln = math.log(sigmod, math.e)
                loss = loss - ln  # 累加值
    
    return loss / n

## 4. 梯度下降
使用随机梯度下降算法，最大化对数后验概率  
学习率gamma和正则化系数lmbda都要取的很小，否则容易计算异常  
每轮迭代完成后，gamma按比例缩小，便于收敛的同时也能避免计算异常

In [13]:
epochs = 10
gamma = 0.001
lmbda = 0.001
loss_list = []
P = user_mat
Q = item_mat
R = rating_mat

n_P = P.size()[0]
n_Q = Q.size()[0]

loss = compute_loss(P, Q, R, pos_neg_list, lmbda)
print('初始后验概率值为 ', loss)

# 计算预测的评分值矩阵
R_pred = torch.matmul(P, Q.t())

for k in range(epochs):  # 多次迭代
    for u in range(1, n_P):  # 针对每个用户的pos-neg item对
        pu = P[u]
        for i in pos_neg_list[u][0]:
            qi = Q[i]
            for j in pos_neg_list[u][1]:
                qj = Q[j]
                xuij = R_pred[u][i] - R_pred[u][j]
                c = 1/(1 + math.exp(xuij))
                
                P[u] += gamma * (c*(Q[i]-Q[j]) + lmbda*P[u])
                Q[i] += gamma * (c*P[u] + lmbda*Q[i])
                Q[j] += gamma * (c*(-P[u]) + lmbda*Q[j])
    loss = compute_loss(P, Q, R, pos_neg_list, lmbda)
    loss_list.append(loss)
    print("k =", k+1, "  loss = ", loss)
    gamma *= 0.6  # 每次迭代后，学习率减小
        

初始后验概率值为  tensor(0.3598)
k = 1   loss =  tensor(0.3478)
k = 2   loss =  tensor(0.3411)
k = 3   loss =  tensor(0.3372)
k = 4   loss =  tensor(0.3349)
k = 5   loss =  tensor(0.3336)
k = 6   loss =  tensor(0.3328)
k = 7   loss =  tensor(0.3323)
k = 8   loss =  tensor(0.3320)
k = 9   loss =  tensor(0.3318)
k = 10   loss =  tensor(0.3317)
